# 1. set WD

In [ ]:
# importar librerías
import os, sys, pandas as pd

# comprobar WD
print(os.getcwd())

# ampliar visualizaciones de pandas
pd.set_option('display.max_rows', None)       # show all rows
pd.set_option('display.max_columns', None)    # show all cols
pd.set_option('display.width', None)          # not restricting width
pd.set_option('display.max_colwidth', None)   # show all content in every cell

# 2.estadísticas de búsqueda

In [ ]:
# importar PubMed Search
pm_df_path = "Anexo_2_carpeta_1_historiales_busqueda/tabla_2.1_PubMed_Search_History.csv"
pm_df = pd.read_csv(pm_df_path, sep=";")

# eliminar cols no útiles
cols = ["Search Details", "Time"]
pm_df = pm_df.drop(columns=cols)

# check
pm_df.head()

In [ ]:
# importar WOS search
wos_df_path = "Anexo_2_carpeta_1_historiales_busqueda/tabla_2.2_WOS_Search_History.csv"
wos_df = pd.read_csv(wos_df_path, sep=";")

# eliminar cols no útiles
cols = ["Entitlements", "Database"]
wos_df = wos_df.drop(columns=cols)

# check
wos_df.head()

## construir tabla de estadísticas de búsqueda

In [ ]:
# construir tabla de stats como dict
search_stats = {
    "Database": ["PubMed", "PubMed Exc.", "WOS", "WOS Exc.", "Excluidos"],
    "#1: términos generales": [pm_df.iloc[0,2], 0, wos_df.iloc[0,1], 0, 0],
    "#2: patologías": [pm_df.iloc[1,2], 0, wos_df.iloc[1,1], 0, 0],
    "#3: genes": [pm_df.iloc[2,2], 0, wos_df.iloc[2,1], 0, 0],
    "#4: ómicas y biología computacional": [pm_df.iloc[3,2], 0, wos_df.iloc[3,1], 0, 0], 
    "#5: biomarcadores": [pm_df.iloc[4,2], 0, wos_df.iloc[4,1], 0, 0], 
    
    "Registros_identificados": [pm_df.iloc[5,2], 0, wos_df.iloc[5,1], 0, 0],
    "Exclusion_cancer": [pm_df.iloc[6,2], 0, wos_df.iloc[6,1], 0, 0],
    "Exclusion_last_10y": [pm_df.iloc[7,2], 0, wos_df.iloc[7,1], 0, 0],
    "Exclusion_ingles": [pm_df.iloc[8,2], 0, wos_df.iloc[8,1], 0, 0] 
}

# pasar a df
search_stats = pd.DataFrame(search_stats).T

# renombrar columnas con valores de la primera fila (menos última col) y dropearla
search_stats.columns = ["PubMed", "PubMed Exc.", "WOS", "WOS Exc.", "Excluidos"]
search_stats = search_stats.iloc[1:]

# castear cols numericas
for col in search_stats.columns[1:]:
    search_stats[col] = search_stats[col].astype(int)

# calcular valores de excluidos
## excluidos pubmed
search_stats.iloc[6, 1] = search_stats.iloc[5, 0] - search_stats.iloc[6, 0]
search_stats.iloc[7, 1] = search_stats.iloc[6, 0] - search_stats.iloc[7, 0]
search_stats.iloc[8, 1] = search_stats.iloc[7, 0] - search_stats.iloc[8, 0]

## excluidos wos
search_stats.iloc[6, 3] = search_stats.iloc[5, 2] - search_stats.iloc[6, 2]
search_stats.iloc[7, 3] = search_stats.iloc[6, 2] - search_stats.iloc[7, 2]
search_stats.iloc[8, 3] = search_stats.iloc[7, 2] - search_stats.iloc[8, 2]

## excluidos total
search_stats.iloc[6, 4] = search_stats.iloc[6, 1] + search_stats.iloc[6, 3]
search_stats.iloc[7, 4] = search_stats.iloc[7, 1] + search_stats.iloc[7, 3]
search_stats.iloc[8, 4] = search_stats.iloc[8, 1] + search_stats.iloc[8, 3]

# trasponoer y checkear
#search_stats = search_stats.T
search_stats

Los excluidos están bien, ahora sigo con el tratamiento

In [ ]:
# eliminar excluidos de cada BBDD
search_stats = search_stats.drop(columns=["PubMed Exc.", "WOS Exc."])

# sumar incluidos de pubmed y wos
search_stats["Total"] = search_stats["PubMed"]+search_stats["WOS"]

# check
search_stats

# 3. deduplicación

In [ ]:
# cargar como dataframe
path1 = "./Anexo_2_carpeta_1_historiales_busqueda/tabla_2.3_deduplication.csv"
df = pd.read_csv(path1, sep=",")
print("\nlongitud pre-dedup: ", df.shape)

# localizar posibles duplicados con "DOI"
df = df.drop_duplicates(subset="DOI")
print("\nlongitud post-dedup: ", df.shape)

In [ ]:
# Exportar lista de artículos no duplicados para revisión manual (si es necesaria) o recarga
df.to_csv('../Anexo_3_tiab_screening/Anexo_3_carpeta_1_inputs/tabla_3.0_articulos_pre_revision.csv', index=False)

## Ahora añado los nºs a la tabla de estadística

In [ ]:
# construir tabla de deduplicaciones totales como dict (para luego concatenar)
dups = search_stats.iloc[-1,-1] - len(df)
dups_df = {"Duplicados y retractados": [0, 0, int(dups), int(len(df))]}

# pasar a df (y trasponer)
dups_df = pd.DataFrame(dups_df).T.astype(int)

# renombrar columnas
dups_df.columns = ["PubMed", "WOS", "Excluidos", "Total"]

# concatenar a search_stats
search_stats = pd.concat([search_stats, dups_df], axis=0)
search_stats

Exporto los resultados

In [ ]:
# Exportar lista de artículos CON abstract para revisión manual (si es necesaria) o recarga
search_stats.to_csv('./Anexo_2_carpeta_2_resultados/tabla_2.4_search_stats.csv', index=True)